In [1]:
from tkinter import *
import cv2 
from PIL import Image, ImageTk 
import numpy as np
import time
import sys
from pyfirmata import Arduino

In [22]:
# Seting up pyfirmata params
board = Arduino('COM10')

# Left motor pins
enableLeftPin = board.get_pin('d:11:p')
input1LeftPin = board.get_pin('d:13:o')
input2LeftPin = board.get_pin('d:12:o')

# Right motor pins
enableRightPin = board.get_pin('d:10:p')
input3RightPin = board.get_pin('d:9:o')
input4RightPin = board.get_pin('d:8:o')


SerialException: could not open port 'COM10': PermissionError(13, 'Access is denied.', None, 5)

In [20]:
# Pruned aruco type dict. See PoseEstimation.ipynb for full list
ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL}

def setMotorSpeeds(leftSpeed, rightspeed):
        # Set left motor direction
        if leftspeed >= 0:
            input1LeftPin.write(1)
            input2LeftPin.write(0)
        else:
            input1LeftPin.write(0)
            input2LeftPin.write(1)
            leftSpeed = -leftSpeed
      
        # Set right motor deirection
        if rightSpeed >= 0:
            input3LeftPin.write(1)
            input4LeftPin.write(0)
        else:
            input3LeftPin.write(0)
            input4LeftPin.write(1)
            rightSpeed = -rightSpeed
    
        enableLeftPin.write(leftSpeed)
        enableRightPin.write(rightSpeed)

# output coords of both 0 and 153
# adjust loop to accommodate
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters,
                                                                cameraMatrix=matrix_coefficients,
                                                                distCoeff=distortion_coefficients)

    if len(corners) > 0:
        for i in range(0, len(ids)):
            if ids[i] in [0, 153]:
                rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                               distortion_coefficients)
                cv2.aruco.drawDetectedMarkers(frame, corners)
                cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
                print(f"ArUco marker ID: {ids[i]}, Coordinates: {tvec}")

    return frame

# Not used (yet?) see ArucoDetect.ipynb for demo
def aruco_display(corners, ids, rejected, image):
    if len(corners) > 0:
        ids = ids.flatten()
        for (markerCorner, markerID) in zip(corners, ids):
            corners = markerCorner.reshape((4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = corners

            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))

            cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
            cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
            cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
            cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)

            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)

            cv2.putText(image, str(markerID), (topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)
            print("[Inference] ArUco marker ID: {}".format(markerID))

    return image

# Function for capturing video frame and displaying it.
# Schedules itself to run again in 10ms
# This will probably be our loop
def open_camera():
    global initiate_cleaning, cleaning, pts, i
    # Capture the video frame by frame 
    _, frame = vid.read()

    # Convert image from one color space to other 
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)

    # Capture the latest frame and transform to image 
    captured_image = Image.fromarray(opencv_image)

    # Convert captured image to photoimage 
    photo_image = ImageTk.PhotoImage(image=captured_image)

    # Displaying photoimage in the label 
    label_widget.photo_image = photo_image

    # Configure image in the label 
    label_widget.configure(image=photo_image)

    # Run the following if the "Clean" button has been clicked
    # Check that the robot is not already cleaning
    if initiate_cleaning and not cleaning:
        pts = generate_path()  # Update to accommodate new func
        i = 0
        initiate_cleaning = False
        cleaning = True

    if cleaning:
        # if dist to pts(i) < threshold
        # if i == length(pts): cleaning = False, motor speeds = 0
        # else: i++, reset error vals
        # run PID
        leftSpeed = 255
        rightSpeed = 255
        setMotorSpeeds(leftSpeed, rightSpeed)

    # Repeat the same process after every 10 seconds 
    label_widget.after(10, open_camera)

# Run once when "Clean" is clicked
def generate_path():
    global pts
    #pts = []


In [21]:
# Setting Aruco params
aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters_create()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))

# Below is an example of how the aruco vid processing was set up 
# refernce for making camera loop

# cap = cv2.VideoCapture(0)

# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)



# while cap.isOpened():
    
#     ret, img = cap.read()
    
#     output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)

#     cv2.imshow('Estimated Pose', output)

#     key = cv2.waitKey(1) & 0xFF
#     if key == ord('q'):
#         break


In [ ]:
# Define a video capture object 
vid = cv2.VideoCapture(0) 

# Declare the width and height in variables 
width, height = 800, 600

# Set the width and height 
# adjust to match phone resolution
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width) 
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height) 

# Create a GUI app 
app = Tk() 

# Bind the app with Escape keyboard to 
# quit app whenever pressed 
app.bind('<Escape>', lambda e: app.quit()) 

# Create a label and display it on app 
label_widget = Label(app) 
label_widget.pack() 

# Create a button to open the camera in GUI app 
button1 = Button(app, text="Open Camera", command=open_camera) 
button1.pack() 

# Create 4 input fields for user define cleaning boundaries
# x1, y1, x2, y2

# Create button to control set bool initiate_cleaning = True

# initialize state as not cleaning
cleaning = True # True for BT test, usually False
initiate_cleaning = False

# bounding box coords 

# initialize array of desired pts and all control variables
pts = []
i = 0


# Create an infinite loop for displaying app on screen 
app.mainloop() 